In [1]:
import os
from dotenv import load_dotenv
import openai
# Load environment variables from .env file
from autogen import  AssistantAgent
from autogen.agentchat.contrib.llamaindex_conversable_agent import LLamaIndexConversableAgent
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, GPTVectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from dotenv import load_dotenv
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.agent import AgentRunner
from llama_index.llms import openai
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.capabilities import teachability
from autogen import Cache
from autogen.agentchat.contrib.capabilities.text_compressors import LLMLingua
from autogen.agentchat.contrib.capabilities.transforms import TextMessageCompressor
from autogen.agentchat.contrib.capabilities import transform_messages
import autogen
import nest_asyncio
from llama_parse import LlamaParse
import Stemmer
from autogen import ConversableAgent, UserProxyAgent
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
    ReActAgentWorker,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.tools import RetrieverTool
from llama_index.core.schema import IndexNode
from llama_index.core.selectors import PydanticMultiSelector

from llama_index.core.retrievers import RouterRetriever,SummaryIndexRetriever,RecursiveRetriever
from llama_index.core import SummaryIndex
from huggingface_hub import login

nest_asyncio.apply()
load_dotenv()


hugging_face_api = os.getenv('HUGGINGFACE_API_KEY')
login(hugging_face_api)

openai_api_key = os.getenv("OPENAI_API_KEY")
LLAMA_CLOUD_KEY = os.getenv("LLAMA_CLOUD_KEY")

#global configs
Settings.text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
Settings.llm = openai.OpenAI()
Settings.embed_model = OpenAIEmbedding()

d:\docs\LLM_RAG_structureddata\autogen\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


resource module not available on Windows


Data Ingestion

In [3]:

# Load and process documents (ensure you have a `documents/` folder)


parser = LlamaParse(
    api_key=LLAMA_CLOUD_KEY,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    verbose=True,
)

file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_dir=r"Data_repo", file_extractor=file_extractor).load_data()

# Create a vector index


Started parsing the file under job_id 0dc3a5e4-afae-4ea6-89c0-368366ecb036
Started parsing the file under job_id 57bb50c5-cf90-4f01-ae7f-cdebce2f7197
Started parsing the file under job_id d470a376-6e37-4222-ad40-9ba5e69e3d80
Started parsing the file under job_id 9e25643f-83b7-41fb-872b-30040661192d
Started parsing the file under job_id 6cde4770-2789-4256-bd00-b9be4ab5fd0c


    
    Creating normal agent from vector index retriever
    -> GPTVectorStoreIndex for indexing
    -> VectorIndexRetriever for retriever
    -> Retriever query engine to convert retriever to query engine
    -> QueryEngineTool to convert gquery engine to tool
    -> create agent using the tool

In [4]:

index = GPTVectorStoreIndex.from_documents(documents, transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=20)])
retriever = VectorIndexRetriever(index=index, similarity_top_k=3)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)
retrieval_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(name="document_retriever", description="Retrieves relevant document sections"),
)


agent = AgentRunner.from_llm([retrieval_tool])

    Creating Structured Planner Agent tool
    -> creating Vector Index Retriever
    -> Creating BM25 Retriever
    -> using them with QueryFusionRetriever to get the best out of the two
    -> building a query engine out of that
    -> converting the query engine into a tool

In [5]:
#First create two different types of retrievers



# 1st is vector store
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=3)

# 2nd is bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=2,
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)


# Next, we can create our fusion retriever, which well return the top-2 most similar nodes from the 4 returned nodes from the retrievers:
query_fusion_retriever = QueryFusionRetriever(
    [vector_retriever, bm25_retriever],
    similarity_top_k=2,
    num_queries=4,  # set this to 1 to disable query generation
    mode="reciprocal_rerank",
    use_async=True,
    # query_gen_prompt="...",  # we could override the query generation prompt here
)


# Creating query engine from the retriever
advanced_retriever_query_engine = RetrieverQueryEngine.from_args(query_fusion_retriever)


# creating tool out of the query engine
advanced_retrieval_tool = QueryEngineTool(
    query_engine=advanced_retriever_query_engine,
    metadata=ToolMetadata(name="advanced_retriever", description="Retrieves relevant document sections for advanced queries"),
)



Segment to create agent worker and StructuredPlannerAgent for advanced search

In [6]:

# create the function calling worker for reasoning
worker = FunctionCallingAgentWorker.from_tools(
    [advanced_retrieval_tool, retrieval_tool]
)

# wrap the worker in the top-level planner
structured_agent = StructuredPlannerAgent(
    worker, tools= [advanced_retrieval_tool, retrieval_tool]
)

In [7]:
# initialize modules
chunk_sizes = [128, 256, 512, 1024]
nodes_list = []
vector_indices = []
for chunk_size in chunk_sizes:
    print(f"Chunk Size: {chunk_size}")
    splitter = SentenceSplitter(chunk_size=chunk_size, chunk_overlap=70)
    nodes = splitter.get_nodes_from_documents(documents)

    # add chunk size to nodes to track later
    for node in nodes:
        node.metadata["chunk_size"] = chunk_size
        node.excluded_embed_metadata_keys = ["chunk_size"]
        node.excluded_llm_metadata_keys = ["chunk_size"]

    nodes_list.append(nodes)

    # build vector index
    vector_index = VectorStoreIndex(nodes)
    vector_indices.append(vector_index)

Chunk Size: 128
Chunk Size: 256
Chunk Size: 512
Chunk Size: 1024


In [8]:
retriever_dict = {}
retriever_nodes = []
for chunk_size, vector_index in zip(chunk_sizes, vector_indices):
    node_id = f"chunk_{chunk_size}"
    node = IndexNode(
        text=(
            "Retrieves relevant context from the Llama 2 paper (chunk size"
            f" {chunk_size})"
        ),
        index_id=node_id,
    )
    retriever_nodes.append(node)
    retriever_dict[node_id] = vector_index.as_retriever()

summary_index = SummaryIndex(retriever_nodes)

recursive_retriever = RecursiveRetriever(
    root_id="root",
    retriever_dict={"root": summary_index.as_retriever(), **retriever_dict},
)

recursive_retriever_query_engine = RetrieverQueryEngine(recursive_retriever)
recursive_retrieval_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(name="document_retriever", description="Retrieves info across all documents"),
)

recursive_retrieval_agent = AgentRunner.from_llm([recursive_retrieval_tool,advanced_retrieval_tool, retrieval_tool])

In [9]:
summary_retriever = SummaryIndexRetriever(index=summary_index, similarity_top_k=6)
vector_query_engine = RetrieverQueryEngine.from_args(vector_retriever)
bm25_query_engine = RetrieverQueryEngine.from_args(bm25_retriever)

vector_retrieval_tool = QueryEngineTool(
    query_engine=vector_query_engine,
    metadata=ToolMetadata(name="vector_document_retriever", description="Retrieves relevant document sections using vector indexes"),
)
bm25_retrieval_tool = QueryEngineTool(
    query_engine=bm25_query_engine,
    metadata=ToolMetadata(name="bm25_document_retriever", description="Retrieves relevant document sections using bm25 retrieval technique"),
)

Open_AI_agent = OpenAIAgent.from_tools([recursive_retrieval_tool, advanced_retrieval_tool, retrieval_tool, vector_retrieval_tool, bm25_retrieval_tool])
all_tools_from_llm_agent = AgentRunner.from_llm([recursive_retrieval_tool, advanced_retrieval_tool, retrieval_tool, vector_retrieval_tool, bm25_retrieval_tool])
all_tools_from_llm_openai_agent = OpenAIAgent.from_llm([recursive_retrieval_tool, advanced_retrieval_tool, retrieval_tool, vector_retrieval_tool, bm25_retrieval_tool])

query = "tell me the names of all the candidates in comma separated format. Don't miss any names"
print("Recursive retriever engine-------------->",recursive_retriever_query_engine.query(query).response)
print("Query fusion engine-------------->",advanced_retriever_query_engine.query(query).response)
print("Vector query engine-------------->",vector_query_engine.query(query).response)
print("BM25 query engine-------------->",bm25_query_engine.query(query).response)
print("GPT Vector retriever query engine-------------->",query_engine.query(query).response)
print("Structured agent-------------->",structured_agent.query(query).response)
print("Recursive query engine-------------->",recursive_retrieval_agent.query(query).response)
print("OpanAI agent from tools---------->",Open_AI_agent.query(query).response)
print("OpanAI agent from LLM---------->",all_tools_from_llm_openai_agent.query(query).response)
print("AgentRunner agent---------->",all_tools_from_llm_agent.query(query).response)


Recursive retriever engine--------------> Kamya, Oindrila Chakravarty
Query fusion engine--------------> Rajarshi Chatterjee, Kamya
Vector query engine--------------> Kamya, Oindrila Chakravarty, Richa Gupta
BM25 query engine--------------> RAJARSHI CHATTERJEE, Oindrila Chakravarty
GPT Vector retriever query engine--------------> Kamya, Oindrila Chakravarty, Richa Gupta
Structured agent--------------> The relevant document sections containing candidate names are:
1. Rajarshi Chatterjee
2. Richa Gupta
Recursive query engine--------------> Kamya, Richa Gupta, Oindrila Chakravarty
OpanAI agent from tools----------> The names of all the candidates are Kamya, Oindrila Chakravarty, Richa Gupta.
OpanAI agent from LLM----------> Kamya, Richa Gupta, Oindrila Chakravarty
AgentRunner agent----------> The candidates for the election are Kamya, Richa Gupta, and Oindrila Chakravarty.


Autogen agent with normal llamaindex agent

In [23]:
# Define LlamaIndexConversableAgent with RAG capabilities


llama_index_agent = LLamaIndexConversableAgent(
    name="LlamaIndexAgent",
    description='llamaindex agent to query information from the vector store',
    llama_index_agent=agent, # Uses LlamaIndex for retrieval
    llm_config={"model": "gpt-4", "api_key": openai_api_key}
)

# Define an AI assistant for responses
# assistant = AssistantAgent(
#     name="AI_Assistant",
#     llm_config={"model": "gpt-4", "api_key": openai_api_key}
# )

human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input,
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower() or None
)

Autogen agent with advanced llamaindex agent

In [24]:
llama_index_agent = LLamaIndexConversableAgent(
    name="LlamaIndexAgent",
    description='llamaindex agent to query information from the vector store',
    llama_index_agent=structured_agent, # Uses LlamaIndex for retrieval
    llm_config={"model": "gpt-4", "api_key": openai_api_key}
)

In [25]:
llm_lingua = LLMLingua()

In [ ]:
with Cache.disk(cache_path_root="/tmp/autogen_cache") as cache:
    
    
    text_compressor = TextMessageCompressor(
        text_compressor=llm_lingua,
        compression_params={"target_token": 13000},
        cache=cache,
    )
    context_handling = transform_messages.TransformMessages(transforms=[text_compressor])
    context_handling.add_to_agent(llama_index_agent)

    result = human_proxy.initiate_chat(llama_index_agent, message='what is the pdf about?', cache=cache)
    result.summary()

human_proxy (to LlamaIndexAgent):

what is the pdf about?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
=== Initial plan ===
Retrieve Relevant Document Sections:
Provide the PDF document -> Relevant sections of the document
deps: []


Advanced Query:
Relevant sections of the document -> Detailed information about the content
deps: ['Retrieve Relevant Document Sections']


> Running step c2936f65-4558-4dbf-b3d2-9968f330ccb6. Step input: Provide the PDF document
Added user message to memory: Provide the PDF document
=== LLM Response ===
Please provide the PDF document that you would like me to analyze.
=== Refined plan ===
Advanced Query:
Relevant sections of the document -> Detailed information about the content
deps: ['Retrieve Relevant Document Sections']


> Running step 2e2318b6-a2f2-4d5d-b038-779ccfba52d6. Step input: Relevant sections of the document
Added user message to memory: Relevant sections of the document
==